In [ ]:
#!/usr/bin/env python3
import argparse
import time
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
from scapy.all import send
from scapy.layers.inet import IP, TCP

In [ ]:
# Функция генерации массива (число пакетов в секунду) по выбранному распределению
def generate_distribution(dist, num_seconds, params):
    if dist == "constant":
        # Для constant просто заполняем массив значением, по умолчанию 1000 п/с.
        value = float(params.get("value", 1000))
        distribution = np.full(num_seconds, value)

    elif dist == "uniform":
        # Равномерное распределение с нижней и верхней границей.
        # Значения по умолчанию: low=800, high=1200 для среднего около 1000.
        low = float(params.get("low", 800))
        high = float(params.get("high", 1200))
        distribution = np.random.uniform(low, high, num_seconds)

    elif dist == "exponential":
        # Экспоненциальное распределение с параметром scale.
        # По умолчанию scale=1000, что задаёт среднее значение 1000.
        scale = float(params.get("scale", 1000))
        distribution = np.random.exponential(scale, num_seconds)

    elif dist == "normal":
        # Нормальное распределение с параметрами mean и std.
        mean = float(params.get("mean", 1000))
        std = float(params.get("std", 100))
        distribution = np.random.normal(mean, std, num_seconds)
        # Исключаем отрицательные значения.
        distribution = np.clip(distribution, 0, None)

    elif dist == "pareto":
        # Распределение Парето.
        # Формула для среднего: mean = (a * xm) / (a - 1), a > 1.
        # Выбираем xm так, чтобы получить среднее ≈ 1000.
        a = float(params.get("a", 2.5))
        xm = 1000 * (a - 1) / a
        distribution = np.random.pareto(a, num_seconds) * xm + xm

    elif dist == "poisson":
        # Распределение Пуассона с параметром lam по умолчанию равным 1000.
        lam = float(params.get("lam", 1000))
        distribution = np.random.poisson(lam, num_seconds)

    elif dist in ["cauchy", "caushy"]:
        # Распределение Коши. Поскольку математическое ожидание не определено,
        # используем сдвиг и масштаб (по умолчанию loc=1000, scale=100)
        loc = float(params.get("loc", 1000))
        scale = float(params.get("scale", 100))
        distribution = np.random.standard_cauchy(num_seconds) * scale + loc
        # Ограничиваем выбросы, чтобы избежать слишком экстремальных значений.
        distribution = np.clip(distribution, 0, 5000)

    elif dist == "gamma":
        # Гамма-распределение: mean = k * theta.
        # По умолчанию k=10, theta=100, что даёт среднее 1000.
        k = float(params.get("k", 10))
        theta = float(params.get("theta", 100))
        distribution = np.random.gamma(k, theta, num_seconds)

    elif dist == "weibull":
        # Распределение Вейбулла. Среднее равно lam * Gamma(1 + 1/a).
        # По умолчанию a=1.5, lam вычисляем так, чтобы среднее ≈ 1000.
        a = float(params.get("a", 1.5))
        gamma_val = sp.gamma(1 + 1 / a)
        lam = float(params.get("lam", 1000 / gamma_val))
        distribution = np.random.weibull(a, num_seconds) * lam

    else:
        raise ValueError("Unsupported distribution type")
    
    # Округляем до целых чисел (число пакетов в секунду) и возвращаем
    distribution = np.rint(distribution).astype(int)
    return distribution

In [3]:
# Функция для построения графика распределения
def plot_distribution(dist_array, distribution_name):
    plt.figure(figsize=(10, 6))
    plt.plot(dist_array, marker="o", linestyle="-", markersize=2)
    plt.title(f"Распределение количества пакетов в секунду: {distribution_name}")
    plt.xlabel("Время (секунды)")
    plt.ylabel("Пакетов в секунду")
    plt.grid(True)
    plt.show()

In [4]:
# Функция отправки TCP-пакетов по заданному количеству в секунду
def send_packets(dst_ip, dst_port, dist_array):
    # Формируем базовый пакет с указанным IP и TCP портом
    packet = IP(dst=dst_ip) / TCP(dport=dst_port)
    for sec, packets in enumerate(dist_array):
        start_time = time.time()
        print(f"Секунда {sec+1}: отправка {packets} пакетов")
        # Отправляем требуемое количество пакетов в эту секунду
        for _ in range(packets):
            send(packet, verbose=False)
        # Ждём до окончания текущей секунды
        elapsed = time.time() - start_time
        if elapsed < 1:
            time.sleep(1 - elapsed)

In [5]:
# Парсинг параметров распределения, переданных в виде строки key=value, ...
def parse_distribution_params(params_str):
    params = {}
    if params_str:
        for item in params_str.split(","):
            key, value = item.split("=")
            params[key.strip()] = value.strip()
    return params

In [ ]:
def main():
    parser = argparse.ArgumentParser(
        description="Генератор TCP-трафика с использованием Scapy и различных распределений (по умолчанию 10 минут)"
    )
    parser.add_argument("dst_ip", help="IP-адрес получателя")
    parser.add_argument("dst_port", type=int, help="TCP-порт получателя")
    parser.add_argument(
        "-d",
        "--duration",
        type=int,
        default=600,
        help="Продолжительность отправки в секундах (по умолчанию 600 секунд = 10 минут)",
    )
    parser.add_argument(
        "-t",
        "--type",
        choices=[
            "constant",
            "uniform",
            "exponential",
            "normal",
            "pareto",
            "poisson",
            "cauchy",
            "gamma",
            "weibull",
        ],
        default="constant",
        help="Тип распределения (по умолчанию constant)",
    )
    parser.add_argument(
        "-p",
        "--params",
        type=str,
        default="",
        help="Параметры распределения в формате key=value,key2=value2 (например: mean=1000,std=100)",
    )
    args = parser.parse_args()

    # Парсим дополнительные параметры распределения
    params = parse_distribution_params(args.params)
    print(f"Используем распределение {args.type} с параметрами: {params}")

    # Генерируем массив значений (число пакетов в секунду) для каждой секунды
    dist_array = generate_distribution(args.type, args.duration, params)
    print("Сгенерирован массив количества пакетов в секунду.")

    # Строим график распределения
    plot_distribution(dist_array, args.type)

    # Отправляем пакеты с заданным количеством в каждую секунду
    print(f"Отправка TCP-пакетов на {args.dst_ip}:{args.dst_port}")
    # send_packets(args.dst_ip, args.dst_port, dist_array)

In [7]:
if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [-d DURATION]
                             [-t {constant,uniform,exponential,normal,pareto,poisson,cauchy,gamma,weibull}]
                             [-p PARAMS]
                             dst_ip dst_port
ipykernel_launcher.py: error: the following arguments are required: dst_ip, dst_port


SystemExit: 2

/home/nemo/anaconda3/envs/py-scapy/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
